<h2>Import packages and install histomics_detect</h2>

In [ ]:
import sys
import tensorflow as tf

#install histomics_detect
!pip install -e /tf/notebooks/histomics_detect

#add to system path
sys.path.append('/tf/notebooks/histomics_detect/')

<h2>Define dataset parameters and create datasets - DCC example</h2>

In [ ]:
#import dataset related packages
from histomics_detect.io import dataset
from histomics_detect.augmentation import crop, flip, jitter, shrink
from histomics_detect.visualization import plot_inference
import numpy as np
import os

#input data path
path = '/tf/notebooks/DCC/data/'

#training parameters
train_tile = 224 #input image size
min_area_thresh = 0.5 # % of object area that must be in random crop to be included
width = tf.constant(train_tile, tf.int32)
height = tf.constant(train_tile, tf.int32)
min_area = tf.constant(min_area_thresh, tf.float32)

#split dataset into training and validation
cases = ['131458', '91315_leica_at2_40x', '135062', '93094',
         '131453', '131450', '135060', '131463', '131459',
         '131440', '131460', '93096', '131449', '131457',
         '131461', '93098', '131447', '93092', '131443',
         '93095', '131448', '93099', '91316_leica_at2_40x', '131462',
         '93091', '135065', '131446', '131441', '101626',
         '93093', '131454', '93097', '131445', '131444',
         '131456', '93090']
id = np.argsort(np.random.rand(len(cases)-1))[0:np.ceil(0.9*len(cases)).astype(np.int32)]
training = [cases[i] for i in id]
validation = list(set(cases).difference(training))

#define parser for filenames
def parser(file):
    name = os.path.splitext(file)[0]
    case = name.split('.')[2]
    roi = '.'.join([name.split('.')[1]] + name.split('.')[-3:])
    return case, roi

#generate training, validation datasets
ds_train_roi = dataset(path, parser, parser, train_tile, training)
ds_validation_roi = dataset(path, parser, parser, 0, validation)

#build training dataset
ds_train_roi = ds_train_roi.map(lambda x, y, z: (*crop(x, y, width, height, 
                                                                 min_area_thresh), z))
ds_train_roi = ds_train_roi.map(lambda x, y, z: (*flip(x, y), z))
ds_train_roi = ds_train_roi.map(lambda x, y, z: (x, jitter(y, 0.05), z))
ds_train_roi = ds_train_roi.map(lambda x, y, z: (x, shrink(y, 0.05), z))
ds_train_roi = ds_train_roi.prefetch(tf.data.experimental.AUTOTUNE)

#build validation datasets
ds_validation_roi = ds_validation_roi.prefetch(tf.data.experimental.AUTOTUNE)

<h2>Create and train detection model - DCC example</h2>

In [ ]:
#import network generation and training packages
from histomics_detect.networks.backbones import pretrained, residual
from histomics_detect.networks.rpns import rpn
from histomics_detect.networks.fast_rcnn import fast_rcnn
from histomics_detect.models.faster_rcnn import FasterRCNN

#choices for anchor sizes - all anchors 1:1 aspect ratio
anchor_sizes = [32, 64, 96] #width/height of square anchors in pixels at input mag.

#feature network parameters
backbone_args = {'name': 'resnet50v2',
                 'stride': 1, #stride (pixels) in first convolution of backbone
                 'blocks': 14} #number of residual blocks to use in backbone

#rpn network parameters
rpn_args = {'kernels': [3], #kernel sizes (receptive fields) for rpn convolutions
            'dimensions': [256], #number of kernels per layer
            'activations': ['relu']} #activation for rpn convolutions

#fast-rcnn network parameters
frcnn_args = {'units': [4096, 4096], #number of units in fast-rcnn dense layers
              'activations': ['relu', 'relu'], #activations for each dense layer
              'pool': 2, #number of tiles to pool during roialign
              'tiles': 3} #number of tiles to split regressed boxes into during roialign

#training parameters
train_args = {'train_shape': (224, 224, 3), #shape of training instances
              'max_anchors': 256, #maximum number of negative anchors to sample per epoch
              'np_ratio': 0.5, #largest ratio of negative : positive anchors per batch
              'lmbda': 10.0} #weighting factor for region-proposal network regression loss

#validation parameters
validation_args = {'tau': 0.5, #objectness threshold used to call positive anchors at inference
                   'nms_iou': 0.3, #min nms threshold used to filter overlapping objects
                   'tpr_iou': 0.5, #single iou threshold used to calculate tpr, fnr during validation
                   'margin': 32, #margin in pixels around image edge to exclude from validation
                   'ap_ious': [0.25, 0.5, 0.75], #ious thresholds to use in evaluating mAP
                   'ap_delta': 0.1} #precision step size for calculating mAP

#create FasterRCNN keras model
model = FasterRCNN(backbone_args, rpn_args, frcnn_args, train_args, validation_args, anchor_sizes)

#compile FasterRCNN model with losses
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss=[tf.keras.losses.BinaryCrossentropy(from_logits=True),
                    tf.keras.losses.Huber()])

#fit FasterRCNN model
model.fit(x=ds_train_roi, batch_size=1, epochs=50, verbose=1,
          validation_data=ds_validation_roi, validation_freq=50)

<h2>Define dataset parameters and create datasets - DLBCL example</h2>

In [ ]:
#import dataset related packages
from histomics_detect.io import dataset, resize
from histomics_detect.augmentation import crop, flip, jitter, shrink
from histomics_detect.visualization import plot_inference
import numpy as np
import os

#input data path
path = '/tf/notebooks/DLBCL/detection/'

#training parameters
train_tile = 224 #input image size
min_area_thresh = 0.5 # % of object area that must be in crop to be included
width = tf.constant(train_tile, tf.int32)
height = tf.constant(train_tile, tf.int32)
min_area = tf.constant(min_area_thresh, tf.float32)

#define filename parsers
def png_parser(png):
    file = os.path.splitext(png)[0]
    case = file.split('.')[0]
    roi = '.'.join(file.split('.')[1:])
    return case, roi

def csv_parser(csv):
    file = os.path.splitext(csv)[0]    
    case = file.split('.')[0]
    roi = '.'.join(file.split('.')[1:2] + file.split('.')[-3:])
    return case, roi

training = ['DCBT_2_CMYC', 'DCBT_3_CMYC', 'DCBT_5_CMYC',
            'DCBT_9_CMYC', 'DCBT_10_CMYC', 'DCBT_12_CMYC', 
            'DCBT_14_CMYC', 'DCBT_18_CMYC', 'DCBT_19_CMYC', 
            'DCBT_20_CMYC', 'DCBT_21_CMYC', 'DCBT_22_CMYC']
validation = ['DCBT_1_CMYC', 'DCBT_4_CMYC', 'DCBT_6_CMYC',
              'DCBT_8_CMYC', 'DCBT_11_CMYC',
              'DCBT_13_CMYC', 'DCBT_15_CMYC', 'DCBT_16_CMYC',
              'DCBT_17_CMYC']


#generate training, validation datasets
ds_train_roi = dataset(path, png_parser, csv_parser, train_tile, training)
ds_validation_roi = dataset(path, png_parser, csv_parser, 0, validation)

#build training dataset
ds_train_roi = ds_train_roi.map(lambda x, y, z: (*resize(x, y, 2.0), z))
ds_train_roi = ds_train_roi.map(lambda x, y, z: (*crop(x, y, width, height, 
                                                                 min_area_thresh), z))
ds_train_roi = ds_train_roi.map(lambda x, y, z: (*flip(x, y), z))
ds_train_roi = ds_train_roi.map(lambda x, y, z: (x, jitter(y, 0.05), z))
ds_train_roi = ds_train_roi.map(lambda x, y, z: (x, shrink(y, 0.05), z))
ds_train_roi = ds_train_roi.prefetch(tf.data.experimental.AUTOTUNE)

#build validation datasets
ds_validation_roi = ds_validation_roi.map(lambda x, y, z: (*resize(x, y, 2.0), z))
ds_validation_roi = ds_validation_roi.prefetch(tf.data.experimental.AUTOTUNE)

<h2>Create and train detection model - DLBCL example</h2>

In [ ]:
#import network generation and training packages
from histomics_detect.networks.rpns import rpn
from histomics_detect.models.faster_rcnn import FasterRCNN

#choices for anchor sizes - all anchors 1:1 aspect ratio
anchor_px = tf.constant([32, 48, 64], dtype=tf.int32) #width/height of square anchors in pixels at input mag.

#feature network parameters
backbone_stride = 1 #strides in feature generation network convolution
backbone_blocks = 14 #number of residual blocks to use in backbone
backbone_dimension = 256 #number of features generated by rpn convolution

#rpn network parameters
rpn_kernel = [3] #kernel size for rpn convolution
rpn_act_conv = ['relu'] #activation for rpn convolutional layers

#anchor filtering parameters
neg_max = 128 #maximum number of negative/positive anchors to keep in each roi
pos_max = 128
rpn_lmbda = 10.0 #weighting for rpn regression loss
roialign_tiles = 3.0 #roialign - number of horizontal/vertical tiles in a proposal
roialing_pool = 2.0 #roialign - number of horizontal/vertical samples in each tile

#create backbone and rpn networks
resnet50 = tf.keras.applications.ResNet50(
    include_top=False, weights='imagenet', input_tensor=None,
    input_shape=(train_tile, train_tile, 3), pooling=None)
rpnetwork, backbone = rpn(resnet50, n_anchors=tf.size(anchor_px),
                          stride=backbone_stride, blocks=backbone_blocks, 
                          kernels=rpn_kernel, dimensions=[backbone_dimension],
                          activations=rpn_act_conv)

#create FasterRCNN keras model
model = FasterRCNN(rpnetwork, backbone, [width, height], anchor_px, rpn_lmbda)

#compile FasterRCNN model with losses
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss=[tf.keras.losses.BinaryCrossentropy(from_logits=True),
                    tf.keras.losses.Huber()])

#fit FasterRCNN model
model.fit(x=ds_train_roi, batch_size=1, epochs=50, verbose=1,
          validation_data=ds_validation_roi, validation_freq=50)

<h2>Inference on a single image - model.call() </h2>

In [ ]:
#generate and visualize thresholded, roialign outputs
data = ds_validation_roi.shuffle(100).take(1).get_single_element()
rgb = tf.cast(data[0], tf.uint8)
regressions = model(rgb, tau=0.5, nms_iou=0.3)
plot_inference(rgb, regressions)

<h2>Raw inference on a single image - model.raw() </h2>

In [ ]:
#generate raw rpn outputs
objectness, boxes, features = model.raw(rgb)

#threshold rpn proposals
boxes_positive, objectness_positive, positive = model.threshold(boxes, objectness, model.tau)
        
#perform non-max suppression on rpn positive predictions
boxes_nms, objectness_nms, selected = model.nms(boxes_positive, objectness_positive, model.nms_iou)
        
#generate roialign predictions for rpn positive predictions
align_boxes = model.align(boxes_nms, features, model.field, model.pool, model.tiles)

#apply thresholding, nms, and roialign
plot_inference(rgb, align_boxes)

<h2>Batch inference using tf.data.Dataset.map </h2>

In [ ]:
#mapping model using data.Dataset.map keeps outputs from different images separate 
map_output = ds_validation_roi.take(5).map(lambda x, y, z: (model(x), y, z))
map_output = [element for element in map_output]

#compare to using model.predict which merges the outputs from all images
predict_output = model.predict(ds_validation_roi.take(5))

<h2>Batch evaluation - model.evaluate() </h2>

In [ ]:
#performance evaluation on multiple images from a tf.data.Dataset
metrics = model.evaluate(ds_validation_roi)

<h2>Save and Load as Keras.Model</h2>

In [ ]:
#compute model output shape to prepare for saving - this shape can be changed after loading
model.compute_output_shape([224, 224, 3])

#save checkpoint
model.save('as_keras')

#load model
restored = tf.keras.models.load_model('as_keras', custom_objects={'FasterRCNN': FasterRCNN}, compile=True)

#check that outputs are same
assert tf.math.reduce_all(tf.math.equal(restored(rgb), model(rgb)))